In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

In [2]:
qs_uni_rank = pd.DataFrame.from_csv('QS_dataframe.csv')

In [ ]:
qs_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508694016501'
qs_r = requests.get(qs_url)

In [ ]:
qs_uni_dict = qs_r.json()

create the data frame

In [ ]:
qs_columns = ['Name','Region','Location','QS Rank','QS Score','No staffs', 'No internat staffs', 'No students', 'No internat students']
qs_uni_rank = pd.DataFrame(index = [], columns=qs_columns)
qs_uni_rank.head()

In [ ]:
def get_numbers_stud_staff(url):
    r_uni = requests.get("https://www.topuniversities.com" + url)
    
    html_request_uni = BeautifulSoup(r_uni.text, 'html.parser')
    
    header_staff = html_request_uni.find('div',class_='faculty-main wrapper col-md-4')
    header_stud_total = html_request_uni.find('div',class_='students-main wrapper col-md-4')
    header_stud_inter = html_request_uni.find('div',class_='int-students-main wrapper col-md-4')
    
    if ((header_staff == None) & (header_stud_total == None) & (header_stud_inter == None)):
        return (-1,-1,-1,-1)
        

    subdiv_staff_total = header_staff.find('div', class_='total faculty')
    subdiv_staff_inter = header_staff.find('div', class_= 'inter faculty')
    subdiv_stud_total = header_stud_total.find('div', class_='total student')
    subdiv_stud_inter = header_stud_inter.find('div', class_='total inter')
    
    num_staff_total = int(subdiv_staff_total.find('div', class_ = 'number').text[1:-1].replace(',',''))
    if (subdiv_staff_inter != None):
        num_staff_inter = int(subdiv_staff_inter.find('div', class_ = 'number').text[1:-1].replace(',',''))
    else:
        num_staff_inter = -1    
    num_stud_total = int(subdiv_stud_total.find('div', class_ = 'number').text[1:-1].replace(',',''))
    num_stud_inter = int(subdiv_stud_inter.find('div', class_ = 'number').text[1:-1].replace(',',''))
    
    return (num_staff_total, num_staff_inter, num_stud_total, num_stud_inter)

In [ ]:
uni_list = qs_uni_dict['data']
i = 0
for uni in uni_list[:200]:
    i+=1
    if (i%40 == 0):
        print(i)
    (tot_staff, inter_staff, tot_stud, inter_stud) = get_numbers_stud_staff(uni['url'])
    uni_df = pd.DataFrame([[uni['title'],uni['region'], uni['country'], uni['rank_display'].replace('=',' '),
                            uni['score'], tot_staff, inter_staff, tot_stud, inter_stud]], columns = qs_columns)
    qs_uni_rank = qs_uni_rank.append(uni_df)

qs_uni_rank.index = list(range(1,201))


In [3]:
qs_uni_rank.to_csv('QS_dataframe.csv')
qs_uni_rank

,Name,Region,Location,QS Rank,QS Score,No staffs,No internat staffs,No students,No internat students
1,Massachusetts Institute of Technology (MIT),North America,United States,1,100.0,2982,1679,11067,3717
2,Stanford University,North America,United States,2,98.7,4285,2042,15878,3611
3,Harvard University,North America,United States,3,98.4,4350,1311,22429,5266
4,California Institute of Technology (Caltech),North America,United States,4,97.7,953,350,2255,647
5,University of Cambridge,Europe,United Kingdom,5,95.6,5490,2278,18770,6699
6,University of Oxford,Europe,United Kingdom,6,95.3,6750,2964,19720,7353
7,UCL (University College London),Europe,United Kingdom,7,94.6,6345,2554,31080,14854
8,Imperial College London,Europe,United Kingdom,8,93.7,3930,2071,16090,8746
9,University of Chicago,North America,United States,9,93.5,2449,635,13557,3379
10,ETH Zurich - Swiss Federal Institute of Techno...,Europe,Switzerland,10,93.3,2477,1886,19815,7563


# Second Website: THE ranking

Note that as the number students, the number of staffs and the number of international students are already mentionned in the QS ranking website they are not scrapped from the Times University Ranking website. We indeed assume that the information provided by the QS ranking website are reliable.

In [4]:
the_r = requests.get("https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json")

the_uni_dict = the_r.json()['data'][:200]

the_columns = ['Name','Location','THE Rank','THE Score']
#the_columns = ['Name','Location','THE Rank','Score', 'No students', 'International students ratio','No students per staff']
the_uni_rank = pd.DataFrame(index = [], columns=the_columns)
the_uni_rank.head()

,Name,Location,THE Rank,THE Score


In [5]:
for uni in the_uni_dict:
    uni_df = pd.DataFrame([[uni['name'],uni['location'], uni['rank'].replace('=',' '),uni['scores_overall']]],columns = the_columns)
    the_uni_rank = the_uni_rank.append(uni_df)
    
the_uni_rank.index = list(range(1,201))

In [8]:
print(qs_uni_rank.columns)
print(the_uni_rank.columns)

Index(['Name', 'Region', 'Location', 'QS Rank', 'QS Score', 'No staffs',
       'No internat staffs', 'No students', 'No internat students'],
      dtype='object')
Index(['Name', 'Location', 'THE Rank', 'THE Score'], dtype='object')


###### Printing the two DataFrames

In [9]:
qs_uni_rank.head()

,Name,Region,Location,QS Rank,QS Score,No staffs,No internat staffs,No students,No internat students
1,Massachusetts Institute of Technology (MIT),North America,United States,1,100.0,2982,1679,11067,3717
2,Stanford University,North America,United States,2,98.7,4285,2042,15878,3611
3,Harvard University,North America,United States,3,98.4,4350,1311,22429,5266
4,California Institute of Technology (Caltech),North America,United States,4,97.7,953,350,2255,647
5,University of Cambridge,Europe,United Kingdom,5,95.6,5490,2278,18770,6699


In [10]:
the_uni_rank.head()

,Name,Location,THE Rank,THE Score
1,University of Oxford,United Kingdom,1,94.3
2,University of Cambridge,United Kingdom,2,93.2
3,California Institute of Technology,United States,3,93.0
4,Stanford University,United States,3,93.0
5,Massachusetts Institute of Technology,United States,5,92.5


# Merging the two DataFrames

In [11]:
# Removing brackets and choosing the option that is less likely to be an acronym
for ind_qs, qs_uni_name in enumerate(qs_uni_rank['Name']):
    if ('(' in qs_uni_name):
        first_brack = qs_uni_name.index('(')
        brack_content = qs_uni_name[first_brack+1:-1]
        if (' ' in brack_content):
            qs_uni_rank['Name'].iloc[ind_qs] = brack_content
        else:
            qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name[:first_brack-1]
         

/usr/local/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


There is only one difference in the location definition between both DataFrames that is fixed now. Indeed it is necessary to do so for the merging operation to not fail.

In [12]:
print("Before changing:")
print(qs_uni_rank[qs_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])
print(the_uni_rank[the_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])
the_uni_rank.loc[194, 'Location'] = 'Russia'
print()
print("After changing:")
print(the_uni_rank[the_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])


Before changing:
95    Russia
Name: Location, dtype: object
194    Russian Federation
Name: Location, dtype: object

After changing:
194    Russia
Name: Location, dtype: object


In [13]:
merged_uni_rank = pd.merge(qs_uni_rank, the_uni_rank, how = "outer")
merged_uni_rank = merged_uni_rank.set_index('Name')
merged_uni_rank.index.is_unique

True

In [14]:
merged_uni_rank.head()

,Region,Location,QS Rank,QS Score,No staffs,No internat staffs,No students,No internat students,THE Rank,THE Score
Name,,,,,,,,,,
Massachusetts Institute of Technology,North America,United States,1.0,100.0,2982.0,1679.0,11067.0,3717.0,5,92.5
Stanford University,North America,United States,2.0,98.7,4285.0,2042.0,15878.0,3611.0,3,93.0
Harvard University,North America,United States,3.0,98.4,4350.0,1311.0,22429.0,5266.0,6,91.8
California Institute of Technology,North America,United States,4.0,97.7,953.0,350.0,2255.0,647.0,3,93.0
University of Cambridge,Europe,United Kingdom,5.0,95.6,5490.0,2278.0,18770.0,6699.0,2,93.2
